#### Importing the libraries

In [219]:
# link :https://archive.ics.uci.edu/ml/datasets/ecoli
import tensorflow as tf
import numpy as np
import csv

#### Importing Data

In [216]:
raw_data = open('ecoli.csv', 'rt')
reader = csv.reader(raw_data, delimiter=',', quoting=csv.QUOTE_NONE)
data = list(reader)
data = np.array(data)
data = data[:,2: -2].astype(float)

In [217]:
print(data)
print(np.shape(data))

[[0.29 0.48 0.5  0.56 0.24]
 [0.4  0.48 0.5  0.54 0.35]
 [0.4  0.48 0.5  0.49 0.37]
 ...
 [0.6  0.48 0.5  0.44 0.39]
 [0.61 0.48 0.5  0.42 0.42]
 [0.74 0.48 0.5  0.31 0.53]]
(336, 5)


#### Data Processing
- 
    Sequences are made into a square image of closest binary number to the the ceiling of the root of the length


In [218]:
rows, columns = np.shape(data)
sqrt_col = int(np.ceil(np.sqrt(columns)))
processed_data = np.zeros([rows, 3, 3])
data_ = np.zeros([rows, 3*3])

for i in range(rows):
#     data[i] = (data[i] - np.mean(data[i]))
    data_[i] = np.append(data[i], np.zeros(3*3 - columns))

    
for i in range(rows):
    for j in range(sqrt_col):
            processed_data[i,j,:] = data_[i, j * 3 : (j+1)*3]

In [195]:
processed_data[0]

array([[0.29, 0.48, 0.5 ],
       [0.56, 0.24, 0.  ],
       [0.  , 0.  , 0.  ]])

#### Generator

In [196]:
def gen_model(input_shape):
    
    Input_1= tf.keras.layers.Input(shape = input_shape)
    
    model = tf.keras.layers.Conv2D(3, (3,3), activation='relu', input_shape=input_shape,kernel_regularizer=tf.keras.regularizers.l2(2e-4), padding ='same')(Input_1)
    model = tf.keras.layers.BatchNormalization()(model)
    
    model = tf.keras.layers.Conv2D(4, (4,4), activation='relu', input_shape=input_shape,kernel_regularizer=tf.keras.regularizers.l2(2e-4), padding ='same')(model)
    model = tf.keras.layers.BatchNormalization()(model)
    
    model = tf.keras.layers.Conv2D(4, (4,4), activation='relu', input_shape=input_shape,kernel_regularizer=tf.keras.regularizers.l2(2e-4), padding ='same')(model)
    model = tf.keras.layers.BatchNormalization()(model)
    
    model = tf.keras.layers.Conv2D(1, (4,4), activation='tanh', input_shape=input_shape,kernel_regularizer=tf.keras.regularizers.l2(2e-4), padding ='same')(model)
    output = tf.keras.layers.BatchNormalization()(model)
    
    model = tf.keras.models.Model(inputs=Input_1, outputs = output, name = 'generator')
    
    return model

In [197]:
gen = gen_model((3,3,1))

In [198]:
gen.summary()

Model: "generator"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_40 (InputLayer)        [(None, 3, 3, 1)]         0         
_________________________________________________________________
conv2d_94 (Conv2D)           (None, 3, 3, 3)           30        
_________________________________________________________________
batch_normalization_94 (Batc (None, 3, 3, 3)           12        
_________________________________________________________________
conv2d_95 (Conv2D)           (None, 3, 3, 4)           196       
_________________________________________________________________
batch_normalization_95 (Batc (None, 3, 3, 4)           16        
_________________________________________________________________
conv2d_96 (Conv2D)           (None, 3, 3, 4)           260       
_________________________________________________________________
batch_normalization_96 (Batc (None, 3, 3, 4)           16

#### Discriminator

In [199]:
def discrim_model(input_shape):
    
    Input_1= tf.keras.layers.Input(shape = input_shape)
    
    model = tf.keras.layers.Conv2D(3, (4,4), activation='relu', input_shape=input_shape,kernel_regularizer=tf.keras.regularizers.l2(2e-4), padding ='same')(Input_1)
    model = tf.keras.layers.BatchNormalization()(model)

    model = tf.keras.layers.Conv2D(3, (2,2), activation='relu', input_shape=input_shape,kernel_regularizer=tf.keras.regularizers.l2(2e-4), padding ='same')(model)
    model = tf.keras.layers.BatchNormalization()(model)
    
    model = tf.keras.layers.Conv2D(3, (2,2), activation='relu', input_shape=input_shape,kernel_regularizer=tf.keras.regularizers.l2(2e-4), padding ='same')(model)
    model = tf.keras.layers.BatchNormalization()(model)

    model = tf.keras.layers.Flatten()(model)
    
    output = tf.keras.layers.Dense(1, activation = 'sigmoid')(model)
    
    model = tf.keras.models.Model(inputs=Input_1, outputs = output, name = 'discriminator')
    
    return model



In [200]:
discrim = discrim_model((3,3,1))

In [201]:
discrim.summary()

Model: "discriminator"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_41 (InputLayer)        [(None, 3, 3, 1)]         0         
_________________________________________________________________
conv2d_98 (Conv2D)           (None, 3, 3, 3)           51        
_________________________________________________________________
batch_normalization_98 (Batc (None, 3, 3, 3)           12        
_________________________________________________________________
conv2d_99 (Conv2D)           (None, 3, 3, 3)           39        
_________________________________________________________________
batch_normalization_99 (Batc (None, 3, 3, 3)           12        
_________________________________________________________________
conv2d_100 (Conv2D)          (None, 3, 3, 3)           39        
_________________________________________________________________
batch_normalization_100 (Bat (None, 3, 3, 3)         

#### Creating the entire network

In [202]:
# Discriminator
optimizer_discrim = tf.keras.optimizers.Adam(0.0002, 0.5)
discriminator = discrim_model((3, 3, 1))
discriminator.compile(loss='binary_crossentropy',optimizer=optimizer_discrim,metrics=['accuracy'])

In [203]:
# Generator
generator = gen_model((3,3,1))

In [204]:
# Combined Network
inp = tf.keras.Input((3,3,1))
gen = generator(inp)
optimizer_gen = tf.keras.optimizers.Adam(0.0002, 0.5)
discriminator.trainable = False
discrim = discriminator(gen)
gan = tf.keras.Model(inp, discrim)
gan.compile(loss='binary_crossentropy', optimizer=optimizer_gen,metrics=['accuracy'])

In [205]:
# Summary
gan.summary()

Model: "model_9"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_44 (InputLayer)        [(None, 3, 3, 1)]         0         
_________________________________________________________________
generator (Model)            (None, 3, 3, 1)           599       
_________________________________________________________________
discriminator (Model)        (None, 1)                 193       
Total params: 792
Trainable params: 575
Non-trainable params: 217
_________________________________________________________________


#### Training

In [206]:
# Training Parameters
batch_size = 336
iterations = 10000

# Training Data
X_train = np.expand_dims(processed_data, axis = 3)

# Labels
true_label = np.ones((batch_size, 1))
fake_label = np.zeros((batch_size, 1))

# Training
for epoch in range(iterations):
    
    # Random sample of data
    random_indices = np.random.randint(0, X_train.shape[0], batch_size)
    discrim_data = X_train[random_indices]
    gaussian = np.random.normal(0.5, 0.25, (batch_size, X_train.shape[1], X_train.shape[2], 1))
    
    # Training the discriminator
      
    discrim_loss_real = discriminator.train_on_batch(discrim_data, true_label)
    generated_data = generator.predict(gaussian)
    discriminator_loss_real = discriminator.train_on_batch(discrim_data, true_label)
    discriminator_loss_fake = discriminator.train_on_batch(generated_data, fake_label)
    discriminator_loss_total = 0.5 * np.add(discrim_loss_real, discriminator_loss_fake)
    
    # Training the gan
    g_loss = gan.train_on_batch(gaussian, true_label)
    
    print('Epoch:', epoch, '| Discrim_l_real:', discriminator_loss_real, '| Discrim_l_fake:', discriminator_loss_fake, '| Gan_l:', g_loss)

Epoch: 0 | Discrim_l_real: [0.47129872, 0.86011904] | Discrim_l_fake: [0.47140706, 0.75] | Gan_l: [0.722258, 0.6577381]
Epoch: 1 | Discrim_l_real: [0.4537199, 0.89285713] | Discrim_l_fake: [0.57652694, 0.57738096] | Gan_l: [0.73360044, 0.6160714]
Epoch: 2 | Discrim_l_real: [0.4355446, 0.9285714] | Discrim_l_fake: [0.7797111, 0.45833334] | Gan_l: [0.7315161, 0.6220238]
Epoch: 3 | Discrim_l_real: [0.42821127, 0.9166667] | Discrim_l_fake: [0.9681279, 0.37202382] | Gan_l: [0.7207465, 0.6547619]
Epoch: 4 | Discrim_l_real: [0.41476953, 0.9464286] | Discrim_l_fake: [0.9939886, 0.35119048] | Gan_l: [0.71739376, 0.66071427]
Epoch: 5 | Discrim_l_real: [0.40668693, 0.94345236] | Discrim_l_fake: [0.9744414, 0.3065476] | Gan_l: [0.74152094, 0.65178573]
Epoch: 6 | Discrim_l_real: [0.39366114, 0.94345236] | Discrim_l_fake: [0.9830702, 0.26488096] | Gan_l: [0.7094457, 0.68154764]
Epoch: 7 | Discrim_l_real: [0.39363658, 0.9404762] | Discrim_l_fake: [1.0058854, 0.24107143] | Gan_l: [0.7024486, 0.6607142

Epoch: 65 | Discrim_l_real: [0.19228606, 1.0] | Discrim_l_fake: [0.36238235, 0.9970238] | Gan_l: [0.5630705, 0.7708333]
Epoch: 66 | Discrim_l_real: [0.19256537, 1.0] | Discrim_l_fake: [0.35745844, 1.0] | Gan_l: [0.5735725, 0.75595236]
Epoch: 67 | Discrim_l_real: [0.18893743, 1.0] | Discrim_l_fake: [0.35572836, 0.9970238] | Gan_l: [0.5840608, 0.7708333]
Epoch: 68 | Discrim_l_real: [0.183904, 1.0] | Discrim_l_fake: [0.3484122, 1.0] | Gan_l: [0.5721185, 0.7291667]
Epoch: 69 | Discrim_l_real: [0.18565348, 1.0] | Discrim_l_fake: [0.35298553, 0.9970238] | Gan_l: [0.55795556, 0.76488096]
Epoch: 70 | Discrim_l_real: [0.18651095, 1.0] | Discrim_l_fake: [0.34573042, 0.9970238] | Gan_l: [0.5442606, 0.7589286]
Epoch: 71 | Discrim_l_real: [0.1863276, 1.0] | Discrim_l_fake: [0.33725458, 1.0] | Gan_l: [0.5528639, 0.7529762]
Epoch: 72 | Discrim_l_real: [0.1859529, 1.0] | Discrim_l_fake: [0.33439606, 0.99404764] | Gan_l: [0.56820154, 0.7380952]
Epoch: 73 | Discrim_l_real: [0.17935425, 1.0] | Discrim_l_

Epoch: 138 | Discrim_l_real: [0.11212751, 1.0] | Discrim_l_fake: [0.19626008, 1.0] | Gan_l: [0.45297062, 0.8244048]
Epoch: 139 | Discrim_l_real: [0.111056626, 1.0] | Discrim_l_fake: [0.1947208, 0.9970238] | Gan_l: [0.43446174, 0.8244048]
Epoch: 140 | Discrim_l_real: [0.113953084, 1.0] | Discrim_l_fake: [0.1895377, 1.0] | Gan_l: [0.4402546, 0.8244048]
Epoch: 141 | Discrim_l_real: [0.11596354, 1.0] | Discrim_l_fake: [0.18708718, 1.0] | Gan_l: [0.42315295, 0.8422619]
Epoch: 142 | Discrim_l_real: [0.11783767, 1.0] | Discrim_l_fake: [0.1864026, 1.0] | Gan_l: [0.41378987, 0.8333333]
Epoch: 143 | Discrim_l_real: [0.11262507, 1.0] | Discrim_l_fake: [0.18943189, 1.0] | Gan_l: [0.4241275, 0.8333333]
Epoch: 144 | Discrim_l_real: [0.11228122, 1.0] | Discrim_l_fake: [0.18445526, 0.9970238] | Gan_l: [0.43342173, 0.81845236]
Epoch: 145 | Discrim_l_real: [0.10759127, 1.0] | Discrim_l_fake: [0.19116718, 0.99404764] | Gan_l: [0.42718127, 0.81845236]
Epoch: 146 | Discrim_l_real: [0.111499906, 1.0] | Disc

Epoch: 208 | Discrim_l_real: [0.075160846, 1.0] | Discrim_l_fake: [0.20300575, 0.9494048] | Gan_l: [0.25745222, 0.97321427]
Epoch: 209 | Discrim_l_real: [0.073712446, 1.0] | Discrim_l_fake: [0.18754742, 0.95238096] | Gan_l: [0.26682726, 0.95238096]
Epoch: 210 | Discrim_l_real: [0.07401153, 1.0] | Discrim_l_fake: [0.18392539, 0.96428573] | Gan_l: [0.27249622, 0.9404762]
Epoch: 211 | Discrim_l_real: [0.07246488, 1.0] | Discrim_l_fake: [0.17102757, 0.9672619] | Gan_l: [0.27254674, 0.9464286]
Epoch: 212 | Discrim_l_real: [0.07144455, 1.0] | Discrim_l_fake: [0.18470202, 0.96428573] | Gan_l: [0.27546892, 0.9345238]
Epoch: 213 | Discrim_l_real: [0.07375814, 1.0] | Discrim_l_fake: [0.17536949, 0.96428573] | Gan_l: [0.26758972, 0.95535713]
Epoch: 214 | Discrim_l_real: [0.07588729, 1.0] | Discrim_l_fake: [0.18255903, 0.96130955] | Gan_l: [0.26558527, 0.9583333]
Epoch: 215 | Discrim_l_real: [0.07233903, 1.0] | Discrim_l_fake: [0.18419993, 0.9583333] | Gan_l: [0.27708673, 0.95238096]
Epoch: 216 | 

Epoch: 275 | Discrim_l_real: [0.055882722, 1.0] | Discrim_l_fake: [0.32448065, 0.875] | Gan_l: [0.17913017, 0.9970238]
Epoch: 276 | Discrim_l_real: [0.0581445, 1.0] | Discrim_l_fake: [0.27743033, 0.9375] | Gan_l: [0.17854282, 1.0]
Epoch: 277 | Discrim_l_real: [0.05506738, 1.0] | Discrim_l_fake: [0.2713875, 0.9375] | Gan_l: [0.18641031, 0.99404764]
Epoch: 278 | Discrim_l_real: [0.05602359, 1.0] | Discrim_l_fake: [0.27488136, 0.9375] | Gan_l: [0.18285559, 0.9970238]
Epoch: 279 | Discrim_l_real: [0.056992438, 1.0] | Discrim_l_fake: [0.28669026, 0.93154764] | Gan_l: [0.20007259, 0.99404764]
Epoch: 280 | Discrim_l_real: [0.055648256, 1.0] | Discrim_l_fake: [0.28040427, 0.9375] | Gan_l: [0.18200439, 0.9970238]
Epoch: 281 | Discrim_l_real: [0.051687505, 1.0] | Discrim_l_fake: [0.28639555, 0.9375] | Gan_l: [0.1817397, 0.9970238]
Epoch: 282 | Discrim_l_real: [0.056042336, 1.0] | Discrim_l_fake: [0.26111057, 0.9464286] | Gan_l: [0.17997462, 0.9970238]
Epoch: 283 | Discrim_l_real: [0.053110503, 1

...

Epoch: 1672 | Discrim_l_real: [0.006096985, 1.0] | Discrim_l_fake: [0.106221385, 1.0] | Gan_l: [2.5059218, 0.0]
Epoch: 1673 | Discrim_l_real: [0.0052850395, 1.0] | Discrim_l_fake: [0.101333424, 1.0] | Gan_l: [2.6278048, 0.0]
Epoch: 1674 | Discrim_l_real: [0.0049842675, 1.0] | Discrim_l_fake: [0.11958695, 1.0] | Gan_l: [2.5682156, 0.0]
Epoch: 1675 | Discrim_l_real: [0.0055160527, 1.0] | Discrim_l_fake: [0.105815865, 1.0] | Gan_l: [2.62022, 0.0]
Epoch: 1676 | Discrim_l_real: [0.0047487956, 1.0] | Discrim_l_fake: [0.097694196, 1.0] | Gan_l: [2.6886814, 0.0]
Epoch: 1677 | Discrim_l_real: [0.0054657273, 1.0] | Discrim_l_fake: [0.09522872, 1.0] | Gan_l: [2.6346405, 0.0]
Epoch: 1678 | Discrim_l_real: [0.0055034016, 1.0] | Discrim_l_fake: [0.100197926, 1.0] | Gan_l: [2.5150623, 0.0]
Epoch: 1679 | Discrim_l_real: [0.005382475, 1.0] | Discrim_l_fake: [0.09875714, 1.0] | Gan_l: [2.7151113, 0.0]
Epoch: 1680 | Discrim_l_real: [0.0049869246, 1.0] | Discrim_l_fake: [0.10418076, 1.0] | Gan_l: [2.74905

Epoch: 1748 | Discrim_l_real: [0.005973204, 1.0] | Discrim_l_fake: [0.04378879, 1.0] | Gan_l: [3.702809, 0.0]
Epoch: 1749 | Discrim_l_real: [0.0056350767, 1.0] | Discrim_l_fake: [0.063913986, 1.0] | Gan_l: [2.938424, 0.0]
Epoch: 1750 | Discrim_l_real: [0.005523501, 1.0] | Discrim_l_fake: [0.06551433, 1.0] | Gan_l: [3.0087337, 0.0]
Epoch: 1751 | Discrim_l_real: [0.0055113453, 1.0] | Discrim_l_fake: [0.047684643, 1.0] | Gan_l: [3.4955328, 0.0]
Epoch: 1752 | Discrim_l_real: [0.0063020275, 1.0] | Discrim_l_fake: [0.0554511, 1.0] | Gan_l: [3.053021, 0.0]
Epoch: 1753 | Discrim_l_real: [0.0065964153, 1.0] | Discrim_l_fake: [0.05094529, 1.0] | Gan_l: [3.193574, 0.0]
Epoch: 1754 | Discrim_l_real: [0.005460408, 1.0] | Discrim_l_fake: [0.06532277, 1.0] | Gan_l: [3.0606492, 0.0]
Epoch: 1755 | Discrim_l_real: [0.0054520057, 1.0] | Discrim_l_fake: [0.05255044, 1.0] | Gan_l: [3.2158673, 0.0]
Epoch: 1756 | Discrim_l_real: [0.0059833876, 1.0] | Discrim_l_fake: [0.06751119, 1.0] | Gan_l: [2.9237578, 0.0

Epoch: 1821 | Discrim_l_real: [0.0061886795, 1.0] | Discrim_l_fake: [0.057387587, 1.0] | Gan_l: [2.3223398, 0.0]
Epoch: 1822 | Discrim_l_real: [0.0069625257, 1.0] | Discrim_l_fake: [0.06216649, 1.0] | Gan_l: [2.3760455, 0.0029761905]
Epoch: 1823 | Discrim_l_real: [0.0056630666, 1.0] | Discrim_l_fake: [0.067120634, 1.0] | Gan_l: [2.3795702, 0.0029761905]
Epoch: 1824 | Discrim_l_real: [0.0066292016, 1.0] | Discrim_l_fake: [0.05487896, 1.0] | Gan_l: [2.31841, 0.008928572]
Epoch: 1825 | Discrim_l_real: [0.006616801, 1.0] | Discrim_l_fake: [0.0597644, 0.9880952] | Gan_l: [2.4528172, 0.020833334]
Epoch: 1826 | Discrim_l_real: [0.006250525, 1.0] | Discrim_l_fake: [0.04565004, 1.0] | Gan_l: [2.655286, 0.008928572]
Epoch: 1827 | Discrim_l_real: [0.00576108, 1.0] | Discrim_l_fake: [0.04448322, 1.0] | Gan_l: [2.3689961, 0.008928572]
Epoch: 1828 | Discrim_l_real: [0.0057256743, 1.0] | Discrim_l_fake: [0.05753139, 1.0] | Gan_l: [2.4161406, 0.0029761905]
Epoch: 1829 | Discrim_l_real: [0.0056432174, 

Epoch: 1893 | Discrim_l_real: [0.0047602654, 1.0] | Discrim_l_fake: [0.078998595, 1.0] | Gan_l: [1.5451893, 0.0029761905]
Epoch: 1894 | Discrim_l_real: [0.0070122406, 1.0] | Discrim_l_fake: [0.085653976, 1.0] | Gan_l: [1.5402277, 0.026785715]
Epoch: 1895 | Discrim_l_real: [0.0067073437, 1.0] | Discrim_l_fake: [0.09025468, 1.0] | Gan_l: [1.6225216, 0.005952381]
Epoch: 1896 | Discrim_l_real: [0.006659139, 1.0] | Discrim_l_fake: [0.09480942, 1.0] | Gan_l: [1.6700914, 0.005952381]
Epoch: 1897 | Discrim_l_real: [0.0057918928, 1.0] | Discrim_l_fake: [0.09673018, 1.0] | Gan_l: [1.6622778, 0.011904762]
Epoch: 1898 | Discrim_l_real: [0.0059952755, 1.0] | Discrim_l_fake: [0.08917055, 1.0] | Gan_l: [1.7686127, 0.0]
Epoch: 1899 | Discrim_l_real: [0.0062391683, 1.0] | Discrim_l_fake: [0.094347745, 1.0] | Gan_l: [1.9939342, 0.0]
Epoch: 1900 | Discrim_l_real: [0.0072141355, 1.0] | Discrim_l_fake: [0.11161062, 1.0] | Gan_l: [1.7873677, 0.0]
Epoch: 1901 | Discrim_l_real: [0.0101718735, 1.0] | Discrim_l

Epoch: 1965 | Discrim_l_real: [0.0155382855, 1.0] | Discrim_l_fake: [0.13481733, 0.9910714] | Gan_l: [2.8801537, 0.011904762]
Epoch: 1966 | Discrim_l_real: [0.015590775, 1.0] | Discrim_l_fake: [0.1144641, 1.0] | Gan_l: [2.4047008, 0.0]
Epoch: 1967 | Discrim_l_real: [0.015554632, 1.0] | Discrim_l_fake: [0.10371145, 0.9970238] | Gan_l: [2.6922357, 0.017857144]
Epoch: 1968 | Discrim_l_real: [0.01843915, 1.0] | Discrim_l_fake: [0.10523837, 0.99404764] | Gan_l: [2.6298318, 0.011904762]
Epoch: 1969 | Discrim_l_real: [0.018883584, 1.0] | Discrim_l_fake: [0.13034353, 0.9970238] | Gan_l: [3.0324287, 0.014880952]
Epoch: 1970 | Discrim_l_real: [0.016242715, 1.0] | Discrim_l_fake: [0.10123487, 0.9970238] | Gan_l: [2.7572107, 0.008928572]
Epoch: 1971 | Discrim_l_real: [0.0112307705, 1.0] | Discrim_l_fake: [0.08437238, 1.0] | Gan_l: [2.782087, 0.026785715]
Epoch: 1972 | Discrim_l_real: [0.018652508, 1.0] | Discrim_l_fake: [0.09621767, 0.9970238] | Gan_l: [2.881712, 0.011904762]
Epoch: 1973 | Discrim

### Statistical inference

#### Statistics of the data

In [207]:
mean_data = np.mean(processed_data, axis = 0)
variance_data = np.mean((processed_data - mean_data )* (processed_data- mean_data ), axis = 0)
skewness_data = np.mean((processed_data - mean_data )* (processed_data- mean_data ) * (processed_data- mean_data ), axis = 0)

In [208]:
print(mean_data)
print('------------')
print(variance_data)
print('------------')
print(skewness_data)

[[0.5        0.49547619 0.5014881 ]
 [0.50002976 0.50017857 0.        ]
 [0.         0.         0.        ]]
------------
[[0.02188512 0.00780811 0.00074183]
 [0.01493125 0.04641009 0.        ]
 [0.         0.         0.        ]]
------------
[[0.00249854 0.00381854 0.00036871]
 [0.00011419 0.00261314 0.        ]
 [0.         0.         0.        ]]


#### Statistics of the generated data

In [212]:
gaussian_noise= np.random.normal(0.5, 0.25, (batch_size, X_train.shape[1], X_train.shape[2], 1))
gen_data = np.squeeze(generator.predict(gaussian))

In [213]:
mean_gen_data = np.mean(gen_data, axis = 0)
variance_gen_data = np.mean((processed_data - mean_gen_data)* (processed_data- mean_gen_data), axis = 0)
skewness_gen_data = np.mean((processed_data - mean_gen_data)* (processed_data- mean_gen_data) * (processed_data- mean_gen_data), axis = 0)

In [214]:
print(mean_gen_data)
print('------------')
print(variance_gen_data)
print('------------')
print(skewness_gen_data)

[[-1.4671538e+00  3.8748941e-01  7.2248483e-01]
 [ 1.0245727e-01  1.8862639e-01  6.6277611e-01]
 [ 1.0929146e+00  2.5235547e-04 -5.1832271e-01]]
------------
[[3.89157914e+00 1.94692517e-02 4.95813887e-02]
 [1.72995135e-01 1.43474847e-01 4.39272175e-01]
 [1.19446228e+00 6.36832843e-08 2.68658428e-01]]
------------
[[ 7.74393598e+00  7.60730238e-03 -1.09165015e-02]
 [ 8.07648026e-02  7.62313658e-02 -2.91139105e-01]
 [-1.30544524e+00 -1.60708253e-11  1.39251763e-01]]
